In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, recall_score, accuracy_score, recall_score, f1_score, precision_score, roc_auc_score 
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
#import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
import pandas as pd
#from sklearn.decomposition import PCA
import xlsxwriter
from IPython.display import display
from sklearn.model_selection import StratifiedKFold

#Traitement du fichier .mat
cancercell = sp.loadmat('breastw.mat')
X = cancercell['X']
y = cancercell['y']
#print(cancercell)
y=np.reshape(y,(683,))


In [ ]:
#Choix hyperparamètres 

HLS=[]
for j in range(4,10):
    for k in range(3,6):
        HLS.append((j,k))
Alpha = [1,0.01, 0.001, 0.0001, 0.00001]
Solver = ['lbfgs','sgd']

#10 fold   
skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset

In [ ]:
skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset
algo=0
test =False
for k in HLS :
    for i in Alpha :
        for j in Solver :
            test =False
            for train_index, test_index in skf.split(X,y):
                #Cross Validation
                X_train, X_test = X[train_index,:], X[test_index,:]
                y_train, y_test = y[train_index], y[test_index]
                y_train=np.ravel(y_train)
                y_test=np.ravel(y_test)
                
                #Implémentation du modèle
                model_MLP = MLPClassifier(hidden_layer_sizes = k, alpha = i, solver = j)
                model_MLP.fit(X_train,y_train)
                y_pred_MLP = model_MLP.predict(X_test)
                                        
                y_scores_MLP = model_MLP.predict_proba(X_test)
                y_scores_MLP = y_scores_MLP[:, 1]
                
                #Calcul score                        
                F1_MLP = round(f1_score(y_test,y_pred_MLP),3)
                
                #Création DataFrame    
                if test == False :
                    matrice_MLP = np.array([F1_MLP])
                    test = True
                else :
                    matrice_MLP = np.append(matrice_MLP,np.array([F1_MLP]).reshape(1,1))

            if algo == 0  :

                df=pd.DataFrame(matrice_MLP,columns=['[{},{},{}]'.format(k,i,j)])

            else :

                df1=pd.DataFrame(matrice_MLP,columns=['[{},{},{}]'.format(k,i,j)])
                df=pd.concat([df, df1], axis = 1)

            algo = algo + 1

display(df) 